# Chat Interface

In [1]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

In [2]:
import os
from mistralai import Mistral

## Panel

[Panel](https://panel.holoviz.org/) is an open source python library that you can use to create dashboards and apps.

In [3]:
import panel as pn
pn.extension()

## Basic Chat UI

In [4]:
def run_mistral(contents, user, chat_interface):
    client = Mistral(api_key=api_key)
    messages = [{"role": "user", "content": contents}]
    chat_response = client.chat.complete(
        model="mistral-large-latest",
        messages=messages)
    return chat_response.choices[0].message.content

In [6]:

chat_interface = pn.chat.ChatInterface(
    callback=run_mistral,
    callback_user="Mistral"
)

chat_interface

BokehModel(combine_events=True, render_bundle={'docs_json': {'7cd1767e-2783-43a7-ad98-43cd1b40f3ca': {'version…

## RAG UI

Below is the RAG code that you used in the previous lesson.

In [7]:
import os
from pathlib import Path
data_dir =Path(os.getcwd()).parent /"data"

In [8]:
import requests
from bs4 import BeautifulSoup
import re

response = requests.get(
    "https://www.deeplearning.ai/the-batch/a-roadmap-explores-how-ai-can-detect-and-mitigate-greenhouse-gases/"
)
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")
tag = soup.find("div", re.compile("^prose--styled"))
text = tag.text
print(text)

# Optionally save this text into a file.
file_name = data_dir/"AI_greenhouse_gas.txt"
with open(file_name, 'w') as file:
    file.write(text)

How can AI help to fight climate change? A new report evaluates progress so far and explores options for the future.What’s new: The Innovation for Cool Earth Forum, a conference of climate researchers hosted by Japan, published a roadmap for the use of data science, computer vision, and AI-driven simulation to reduce greenhouse gas emissions. The roadmap evaluates existing approaches and suggests ways to scale them up.How it works: The roadmap identifies 6 “high-potential opportunities”: activities in which AI systems can make a significant difference based on the size of the opportunity, real-world results, and validated research. The authors emphasize the need for data, technical and scientific talent, computing power, funding, and leadership to take advantage of these opportunities.Monitoring emissions. AI systems analyze data from satellites, drones, and ground sensors to measure greenhouse gas emissions. The European Union uses them to measure methane emissions, environmental orga

In [9]:
import numpy as np
import faiss

client = Mistral(
    api_key=os.getenv("MISTRAL_API_KEY")
)

prompt = """
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=input)
    return embeddings_batch_response.data[0].embedding

def run_mistral(user_message, model="mistral-large-latest"):
    messages = [{"role": "user", "content": user_message}]
    chat_response = client.chat.complete(model=model, messages=messages)
    return chat_response.choices[0].message.content

def answer_question(question, user, instance):
    text = file_input.value.decode("utf-8")

    # split document into chunks
    chunk_size = 2048
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    # load into a vector database
    text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
    d = text_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(text_embeddings)
    # create embeddings for a question
    question_embeddings = np.array([get_text_embedding(question)])
    # retrieve similar chunks from the vector database
    D, I = index.search(question_embeddings, k=2)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    # generate response based on the retrieved relevant text chunks
    response = run_mistral(
        prompt.format(retrieved_chunk=retrieved_chunk, question=question)
    )
    return response

### Connect the Chat interface with your user-defined function

- Note, you can find some sample text files to upload to this RAG UI by clicking on the 'Jupyter' logo and to view the file directory of the lesson.
- Or you can create any text file and copy-paste some text from a web article.

In [10]:
file_input = pn.widgets.FileInput(accept=".txt", value="", height=50)

chat_interface = pn.chat.ChatInterface(
    callback=answer_question,
    callback_user="Mistral",
    header=pn.Row(file_input, "### Upload a text file to chat with it!"),
)
chat_interface.send(
    "Send a message to get a reply from Mistral!",
    user="System",
    respond=False
)
chat_interface

BokehModel(combine_events=True, render_bundle={'docs_json': {'4565e76f-f0d6-4044-9136-a9421e79dafe': {'version…

#### Note about Panel
If you are running this notebook on your local machine:
- After uploading a local text file, and when typing into the textbox for the RAG UI, you may notice that the screen "jumps" downward, and a new code cell is inserted above the chat UI.
- The letter "a" is a jupyter notebook shortcut that inserts a new code cell "above" the current one.
- If you see this, please use your mouse cursor to click back into the "Send a message" textbox and continue typing.  After that, you will not see the screen jump even if you type a letter "a" or any other jupyter notebook shortcut key.

For a more permanent fix, please upgrade to the latest version of juypter lab and panel.